In [ ]:
project='tpcds'

In [38]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import boto3
from io import StringIO
from datetime import date


from s3fs import S3FileSystem

s3_resource = boto3.resource('s3',
                    endpoint_url='http://minio:9000',
                    aws_access_key_id='minio',
                    aws_secret_access_key='minio123',
                    region_name='us-east-1')

s3 = S3FileSystem(key='minio', secret='minio123', client_kwargs={'endpoint_url':'http://minio:9000'})

In [20]:
pd.set_option('display.max_columns', None)
source_bucket = 'source-data'
target_bucket = 'persist-data'

In [21]:
bucket = 'persist-data'
key = 'reference'
file = 'date_dim'


datedim_df = pq.ParquetDataset('s3://{0}/{1}/{2}'.format(bucket,key,file), filesystem=s3).read_pandas().to_pandas()

In [22]:
bucket = 'persist-data'
key = 'tpcds'
file = 'event='

storesales_df = pq.ParquetDataset('s3://{0}/{1}/event={2}'.format(bucket,key,'store_sales'), filesystem=s3).read_pandas().to_pandas()
catalogsales_df = pq.ParquetDataset('s3://{0}/{1}/event={2}'.format(bucket,key,'catalog_sales'), filesystem=s3).read_pandas(columns=['cs_sold_date_sk', 'cs_sold_time_sk', 'cs_item_sk', 'cs_bill_customer_sk', 'cs_bill_cdemo_sk', 'cs_bill_hdemo_sk', 'cs_bill_addr_sk', 'cs_warehouse_sk', 'cs_promo_sk', 'cs_order_number', 'cs_quantity', 'cs_wholesale_cost', 'cs_list_price', 'cs_sales_price', 'cs_ext_discount_amt', 'cs_ext_sales_price', 'cs_ext_wholesale_cost', 'cs_ext_list_price', 'cs_ext_tax', 'cs_coupon_amt', 'cs_net_paid', 'cs_net_paid_inc_tax', 'cs_net_profit']).to_pandas()
websales_df = pq.ParquetDataset('s3://{0}/{1}/event={2}'.format(bucket,key,'web_sales'), filesystem=s3).read_pandas(columns=['ws_sold_date_sk', 'ws_sold_time_sk', 'ws_item_sk', 'ws_bill_customer_sk', 'ws_bill_cdemo_sk', 'ws_bill_hdemo_sk', 'ws_bill_addr_sk', 'ws_warehouse_sk', 'ws_promo_sk', 'ws_order_number', 'ws_quantity', 'ws_wholesale_cost', 'ws_list_price', 'ws_sales_price', 'ws_ext_discount_amt', 'ws_ext_sales_price', 'ws_ext_wholesale_cost', 'ws_ext_list_price', 'ws_ext_tax', 'ws_coupon_amt', 'ws_net_paid', 'ws_net_paid_inc_tax', 'ws_net_profit']).to_pandas()


In [23]:
item_df = pq.ParquetDataset('s3://{0}/{1}/event={2}'.format(bucket,key,'item'), filesystem=s3).read_pandas().to_pandas()
item_df = item_df.loc[item_df['i_category'].notnull()]

In [24]:
sales_df_cols = ['ss_sold_date_sk', 'ss_sold_time_sk', 'ss_item_sk', 'ss_customer_sk', 'ss_cdemo_sk', 'ss_hdemo_sk', 'ss_addr_sk', 'ss_store_sk', 'ss_promo_sk', 'ss_ticket_number', 'ss_quantity', 'ss_wholesale_cost', 'ss_list_price', 'ss_sales_price', 'ss_ext_discount_amt', 'ss_ext_sales_price', 'ss_ext_wholesale_cost', 'ss_ext_list_price', 'ss_ext_tax', 'ss_coupon_amt', 'ss_net_paid', 'ss_net_paid_inc_tax', 'ss_net_profit','ss_type']

In [25]:
## copy sales_df to total_sales_df

total_sales_df = storesales_df.copy()
total_sales_df['ss_type'] = 'store'

In [27]:
websales_df['ss_type'] ='web'
web_sales_df = pd.DataFrame(data=websales_df.values, columns=total_sales_df.columns)

In [28]:
catalogsales_df['ss _type'] ='catalog'
catalog_sales_df = pd.DataFrame(data=catalogsales_df.values, columns=total_sales_df.columns)

In [29]:
## vertical_stack = pd.concat([survey_sub, survey_sub_last10], axis=0)

all_sales_df = pd.concat([total_sales_df, web_sales_df, catalog_sales_df], axis=0)

In [30]:
df = pd.merge(all_sales_df, datedim_df[['d_date_sk', 'd_date', 'd_year', 'd_moy']], left_on='ss_sold_date_sk', right_on='d_date_sk').drop(['ss_sold_date_sk', 'd_date_sk', 'ss_sold_time_sk'], axis=1)

In [31]:
df2 = pd.merge(df, item_df[['i_item_sk', 'i_class', 'i_category']], left_on='ss_item_sk', right_on='i_item_sk').drop(['i_item_sk'], axis=1)
df2.sort_values(by=['ss_customer_sk'], inplace=True)
df3 = df2.groupby(['ss_customer_sk','ss_type','d_year', 'd_moy', 'd_date']).agg({'i_category':(lambda x: ','.join(x)), 'ss_sales_price': sum, 'ss_item_sk': "count"}).reset_index()

In [33]:
columns = ['Music', 'Women','Home','Children','Men','Electronics','Jewelry','Sports','Books','Shoes']


for column in columns:
    df3[column] = df3['i_category'].str.count(column)
    
df3.drop(['i_category'], axis=1, inplace=True)

In [34]:
df3['d_date'] = df3['d_date'] + 'T00:00:00.000Z'

In [36]:
ts = pd.to_datetime('today').strftime('%Y%m%d')

In [39]:

table = pa.Table.from_pandas(df3)
pq.write_to_dataset(table, 's3://{0}/{1}/event={2}_{3}'.format('consumption-data','tpcds','sales_summary',ts), filesystem=s3, compression='snappy')
